# Spine Segmentation

In [ ]:
# import libs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from random import randint
from PIL import Image
import os
import pydicom

import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint

In [ ]:
# Init global variables
DATA_DIR = os.path.abspath('../Data')
X_FILES = []
Y_FILES = []

RESULTS = os.path.abspath('../Results')

IMG_SHAPE = ()
VALIDATION_NUM = 6
BATCH_SIZE = 16

## DataSet

In [ ]:
class Data_train_generator(Sequence):
    def __init__(self, x_files_list: list, y_files_list: list, batch_size, shuffle = True) -> None:
        self.data = x_files_list
        self.labels = y_files_list
        self.batch_size = batch_size

        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def __open_png_y(self, file_path):
        img = Image.open(file_path)
        data = np.array(img, dtype="float32")
        img.close()
        return np.expand_dims(data, axis=-1)

    def __open_dcm_x(self, file_path):
        dcm = pydicom.dcmread(file_path)

        d = dcm.pixel_array.astype("float32") / 255

        return np.expand_dims(d, axis=-1)

    def __getitem__(self, index):
        batch_x = np.array(list(map(self.__open_dcm_x, self.data[index * self.batch_size: (index + 1) * self.batch_size])))
        batch_y = np.array(list(map(self.__open_png_y, self.labels[index * self.batch_size: (index + 1) * self.batch_size])))

        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            for i in range(len(self.data)):
                ind_from, ind_to = randint(0, len(self.data) - 1), randint(0, len(self.data) - 1)
                self.data[ind_from], self.data[ind_to] = self.data[ind_to], self.data[ind_from]
                self.labels[ind_from], self.labels[ind_to] = self.labels[ind_to], self.labels[ind_from]

## Training

In [ ]:
# training
def make_dice_loss(smooth=1e-6, gama=2):
    def dice_loss(y_true, y_pred):
        y_true, y_pred = tf.cast(y_true, dtype=tf.float32), tf.cast(y_pred, dtype=tf.float32)
        nominator = 2 * tf.reduce_sum(tf.multiply(y_pred, y_true)) + smooth
        denominator = tf.reduce_sum(y_pred ** gama) + tf.reduce_sum(y_true ** gama) + smooth
        return 1 - tf.divide(nominator, denominator)
    return dice_loss

data_gen = Data_train_generator(X_FILES[:-VALIDATION_NUM], Y_FILES[:-VALIDATION_NUM], BATCH_SIZE)
val_gen = Data_train_generator(X_FILES[-VALIDATION_NUM:], Y_FILES[-VALIDATION_NUM:], BATCH_SIZE)

model_checkpoint = ModelCheckpoint(
    save_best_only=True,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    filepath=os.path.join(RESULTS, 'saved_weights/{epoch}-weights_unetpp.hdf5')
)

from model import Nest_Net

model_unet = Nest_Net(IMG_SHAPE[0], IMG_SHAPE[1], color_type=1, num_class=1, deep_supervision=True)

model_unet.compile(optimizer='Adam', loss={
    'output_1': make_dice_loss(),
    'output_2': make_dice_loss(),
    'output_3': make_dice_loss(),
    'output_4': make_dice_loss()
}, loss_weights=[1.0, 1.0, 1.0, 1.0])

history_unet = model_unet.fit(x=data_gen, epochs=20, validation_data=val_gen, callbacks=[model_checkpoint])

## Testing

In [ ]:
# testing
TEST_IMG_PATH = ""
TEST_TRUE_IMG_PATH = ""

img = Image.open(TEST_IMG_PATH)
data = np.array(img, dtype="float32")
img.close()
test_img = np.expand_dims(data, axis=-1)

dcm = pydicom.dcmread(TEST_TRUE_IMG_PATH)
d = dcm.pixel_array.astype("float32") / 255
test_true_img = np.expand_dims(d, axis=-1)

res = model_unet.predict(test_img, batch_size=1)
res = (res[0] + res[1] + res[2] + res[3]) / 4
res = res.reshape(IMG_SHAPE)

fig, ax = plt.subplots(nrows=1, ncols=4)
fig.set_figwidth(12)
ax[3].set_title('mask')
ax[1].set_title('prediction')

sns.heatmap(res, ax=ax[0])
sns.heatmap(np.where(res > 0.5, 1, 0), ax=ax[1])
sns.heatmap(test_img.reshape(IMG_SHAPE), ax=ax[2])
sns.heatmap(test_true_img.reshape(IMG_SHAPE), ax=ax[3])
plt.show()